In [13]:
# Import necessary libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [14]:
# Get data from Wikipedia Link
link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_page= requests.get(link).text

# Use Beautiful Soup to parse the HTML/XML codes.
soup = BeautifulSoup(wiki_page,'xml')

#I won´t run the code due to the length of it.
#print(soup.prettify())

In [15]:
table = soup.find('table')

#Create three initial lists
Postcode      = []
Borough       = []
Neighbourhood = []

#I won´t print the table because of the length
#print(table)

# Extract the data  form of the table. 
#'tr' means rows and 'td' means columns:
for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)


In [16]:
#When there are more than 1 Neighborhood: Integrate Postal Codes
unique_p = set(Postcode)
print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

num of unique Postal codes: 84


In [17]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.to_csv('toronto_part1.csv')
df_toronto.head(10)

,Borough,Neighbourhood,Postcode
0,Central Toronto,"Deer Park, Rathnelly, South Hill",M4V
1,West Toronto,Dovercourt Village,M6H
2,North York,Bedford Park,M5M
3,Etobicoke,Islington Avenue,M9A
4,York,Weston,M9N
5,Scarborough,Birch Cliff,M1N
6,Central Toronto,Forest Hill North,M5P
7,Scarborough,Cedarbrae,M1H
8,Central Toronto,Moore Park,M4T
9,Etobicoke,"Beaumond Heights, Jamestown, Mount Olive, Silv...",M9V


In [18]:
df_toronto.shape

(84, 3)